# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled. 
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import Counter
import os
import gc
import random

In [ ]:
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {
        'train': T.Compose([
        T.Resize((256),  # Resize the and smooth images
        interpolation=T.InterpolationMode.BICUBIC),
        T.RandomHorizontalFlip(),    # Randomly flip the image horizontally
        T.RandomRotation(15),  # Randomly rotate the image by up to 15 degrees
        T.RandomResizedCrop(224),          # random crop to 224
        T.ToTensor(),                # Convert the image to a tensor
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
        'valid': T.Compose([
        T.Resize(256),               # Resize the image to 256
        T.CenterCrop(224),          # Center crop to 224
        T.ToTensor(),                # Convert the image to a tensor
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
    ]),
    'test': T.Compose([
        T.Resize(256),               # Resize the image to 256
        T.CenterCrop(224),          # Center crop to 224
        T.ToTensor(),                # Convert the image to a tensor
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
    ])
}

# TODO: Load the datasets with ImageFolder
image_datasets = {
            'train': ImageFolder(train_dir, data_transforms['train']),
            'valid': ImageFolder(valid_dir, data_transforms['valid']),
            'test': ImageFolder(test_dir, data_transforms['test'])
}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=16,shuffle=True, num_workers = os.cpu_count() if torch.cuda.is_available() else 0, pin_memory=True if torch.cuda.is_available() else False),
    'valid': DataLoader(image_datasets['valid'], batch_size=16, shuffle=False, num_workers = os.cpu_count() if torch.cuda.is_available() else 0, pin_memory=True if torch.cuda.is_available() else False),
    'test': DataLoader(image_datasets['test'], batch_size=16, shuffle=False, num_workers = os.cpu_count() if torch.cuda.is_available() else 0, pin_memory=True if torch.cuda.is_available() else False)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# TODO: Build and train your network
model = models.vgg16(pretrained=True)

# Freeze the parameters
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier
num_classes = 102
model.classifier.append(torch.nn.Linear(1000, 256))
model.classifier.append(torch.nn.ReLU())
model.classifier.append(nn.Dropout(p = 0.2))
model.classifier.append(torch.nn.Linear(256, num_classes))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
# Init scheduler Lr
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)

num_epochs = 10  # Set the number of epochs
batch_size = 16  # Set the batch size according to dataloaders
# Move the model to the GPU if available
model.to(device)
model.train()
best_val_loss = float('inf')
early_stop_patience = 5
early_stop_counter = 0
for epoch in range(num_epochs):
    running_loss = 0.0
    num_batches = len(dataloaders['train'])
    train_loader = dataloaders['train']
    valid_loader = dataloaders['valid']

    try:
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
            # Move data to the same device as the model (CPU or GPU)
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()  # Clear previous gradients
            output = model(images)  # Pass flattened features to the classifier
            output = output.view(output.size(0), -1)
            loss = criterion(output, labels)  # Calculate loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            running_loss += loss.item()
            gc.collect()
        # Calculate average loss for the epoch
        avg_loss = running_loss / num_batches
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

        # Step the scheduler with the average loss
        scheduler.step(avg_loss)

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        val_accuracy = 0.0
        
        with torch.no_grad():  # Disable gradient calculation for validation
            for images, labels in tqdm(valid_loader, desc='Validating', unit='batch'):
                images, labels = images.to(device), labels.to(device)
                output = model(images)  # Pass flattened features to the classifier
                output = output.view(output.size(0), -1) 
                loss = criterion(output, labels)  # Calculate validation loss
                val_loss += loss.item()
                # Calculate accuracy
                _, preds = torch.max(output, 1)
                val_accuracy += (preds == labels).sum().item()

        # Average validation loss and accuracy after the loop
        avg_val_loss = val_loss / len(valid_loader)
        avg_val_accuracy = val_accuracy / len(valid_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.2f}, Validation Accuracy: {avg_val_accuracy:.2f}')
      
        gc.collect()
        if avg_val_loss < best_val_loss:
          best_val_loss = avg_val_loss
          early_stop_counter = 0
          # Save best epoch
          torch.save(model.state_dict(), 'my_model.pth')
          print(f'Modelo salvo com Validation Loss: {avg_val_loss:.4f}')
        else:
          early_stop_counter += 1

        if early_stop_counter >= early_stop_patience:
          print("Early stopping acionado")
          break
       
    except Exception as e:
      print(f"An error occurred: {e}")
print(f"Finished training!")

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
def test_model_and_confusion_matrix(model, dataloader, criterion, top_k=10):
    model.eval() 
    test_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # top_k 
            _, preds = torch.topk(outputs, top_k, dim=1)
            total += labels.size(0)
            
            correct += (preds[:, 0] == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds[:, 0].cpu().numpy())  # highest prob

    avg_test_loss = test_loss / len(dataloader)
    accuracy = 100 * correct / total
    gc.collect()
    print(f'Test Loss: {avg_test_loss:.2f}, Test Accuracy: {accuracy:.2f}%')
       
    return avg_test_loss, accuracy, all_labels, all_preds

# Call to action
avg_test_loss, test_accuracy, y_true, y_pred = test_model_and_confusion_matrix(model, dataloaders['test'], criterion, top_k=10)

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
torch.save(model.state_dict(), "my_model.pth")
class_to_idx = image_datasets['train'].class_to_idx  # Access class_to_idx from the dataset
# Save the model and include class_to_idx in the checkpoint
model.class_to_idx = class_to_idx

# After training your model
checkpoint = {
    'epoch': num_epochs,  # Save the number of epochs
    'model_state_dict': model.state_dict(),  # Save the model's state
    'optimizer_state_dict': optimizer.state_dict(),  # Save the optimizer's state
    'class_to_idx': class_to_idx  # Save class to index mapping
}

# Save the checkpoint
torch.save(checkpoint, 'model_checkpoint.pth')
print(checkpoint.items())

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Rebuilding the model
model = models.vgg16(pretrained=True)
checkpoint = torch.load('model_checkpoint.pth', map_location=device)

# Repeat the model params
num_classes = 102
model.classifier.append(torch.nn.Linear(1000, 256))
model.classifier.append(torch.nn.ReLU())
model.classifier.append(nn.Dropout(p = 0.2))
model.classifier.append(torch.nn.Linear(256, num_classes))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
# Init scheduler Lr
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
model.load_state_dict(checkpoint['model_state_dict'])
model.class_to_idx = checkpoint['class_to_idx']
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    if isinstance(image, torch.Tensor):
        image = T.ToPILImage()(image)  # Converter tensor para imagem PIL

    preprocess = T.Compose([
        T.Resize(224),
        T.CenterCrop(224),  # Recorta o centro 224x224 pixels
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = preprocess(image)
    #image = np.asarray(Image.open(image_path), dtype=np.float32)
    image = image.unsqueeze(0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image.to(device)
    return image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
def get_random_image(parent_dir):
    # Get all subdirectories in the parent directory
    subdirectories = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]
    if not subdirectories:
        raise ValueError("No subdirectories found in the specified directory.")

    # Select a random subdirectory
    random_subdirectory = random.choice(subdirectories)
    random_dir_path = os.path.join(parent_dir, random_subdirectory)
    print(f'Selected Random Directory: {random_dir_path}')

    # Select a random image file from the selected subdirectory
    image_files = [f for f in os.listdir(random_dir_path) if f.endswith(('.jpg', '.png'))]
    if not image_files:
        raise ValueError("No image files found in the selected directory.")

    random_image_file = random.choice(image_files)

    # Construct the full path to the image
    image_path = os.path.join(random_dir_path, random_image_file)

    # Open the image
    image = Image.open(image_path)#np.asarray(Image.open(image_path)#, dtype=np.float32)

    return image, image_path  # Return the image and its path

In [ ]:
dataset = test_dir  # Replace with your test directory
image, image_path = get_random_image(test_dir)

# Print the original image size
#width, height = image.size
#print(f'Image size: {width} x {height} pixels')

# Process the image
processed_img = process_image(image)
print(processed_img.shape)
# Display the processed image
processed_img = processed_img.squeeze()
#imshow(processed_img)  # Remover a dimensão extra para exibir corretamente
print(image_path)
# Check the shape of the processed image
# Note: If processed_img is still a PIL image, you need to convert it to a NumPy array to check the shape
if isinstance(processed_img, Image.Image):
    processed_img = np.array((processed_img), dtype=np.float32)  # Convert to NumPy array if needed
    print(processed_img)
imshow(processed_img)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    dataset = image_path  # Replace with your test directory
    image, image_path = get_random_image(test_dir)

    # Process the image
    processed_img = process_image(image)
    print(processed_img.shape)
    # Display the processed image
    #processed_img = processed_img.squeeze()
    processed_img = processed_img.to('cuda' if torch.cuda.is_available() else 'cpu')
    #imshow(processed_img)  # Remover a dimensão extra para exibir corretamente
    print(image_path)
    # Colocar o modelo em modo de avaliação
    model.eval()
    with torch.no_grad():
        # Passar a imagem pelo modelo
        output = model(processed_img)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(topk, dim=1)

    # Converter índices para classes
    idx_to_class = {v: k for k, v in model.class_to_idx.items()}
    top_classes = [idx_to_class[idx.item()] for idx in top_class[0]]
    top_flowers = [cat_to_name[idx_to_class[idx.item()]] for idx in top_class[0]]
    return top_p[0].tolist(), top_classes, top_flowers

In [ ]:
top_probs, top_classes, top_flowers = predict(image_path, model, topk=5)
print(f'Caminho da Imagem: {image_path}')
print("Top 5 Probabilidades:", top_probs)
print("Top 5 Classes:", top_classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def plot_probabilities(image_path, model, topk=5):
    ''' Plotar as probabilidades das 5 classes mais prováveis como um gráfico de barras, junto com a imagem de entrada. '''
    top_probs, top_classes, top_flowers = predict(image_path, model, topk=5)

    ax = imshow(processed_img, title=top_flowers[0])

    # Gráfico de barras
    fig, ax = plt.subplots()
    y_pos = np.arange(len(top_flowers))
    ax.barh(y_pos, top_probs, align='center')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top_flowers)
    ax.invert_yaxis()  # Inverter a ordem das labels
    ax.set_xlabel('Probabilidade')
    plt.show()
    
# Plotar as probabilidades das 5 classes mais prováveis
plot_probabilities(image_path, model, topk=5)